<h1 style="text-align:center;width:100%;background-color:#242424;padding-left:10px;margin-bottom:0px; color:skyblue"><font size = 5 color=skyblue>Segmenting and Clustering Neighborhoods in <br /> Torento</font></h1>
<h3 style="text-align:left;width:100%;background-color:#ddd;padding-left:10px;;margin-top:0px;"> <font size=1 color=#444>Developed by <font size=1 color=green>Reza Sadeghi Jafari </font>- March 2021 </font></h3>

# Introduction

This code developed in regard of Coursera assignment on course "Applied Data Science Capstone". It's required to explore, segment, and cluster the neighborhoods in the city of Toronto based on the postalcode and borough information. The neighborhood data for torento is not readily available on the internet.

"For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

Start by creating a new Notebook for this assignment.
Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas  dataframe like the one shown below:

3. To create the above dataframe:

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.
If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
4. Submit a link to your Notebook on your Github repository.(10 marks)

Note: There are different website scraping libraries and packages in Python. For scraping the above table, you can simply use pandas  to read the table into a pandas dataframe.

Another way, which would help to learn for more complicated cases of web scraping is using the BeautifulSoup package. Here is the package's main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/

Use pandas, or the BeautifulSoup package, or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe."

In [174]:
import pandas as pd
import numpy as np
import folium as fl
import geopy
from geopy.geocoders import Nominatim 
import json 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

In [247]:
# I prefer to use pandas to fetch data from grid in HTML file, it's so fast and easy.
# I change the url to read from history, it seems changed during this month
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1012118802'
dfs = pd.read_html(url)
df=dfs[0]   # it's retrive the first dataframe from all data frames which red by the "read_html" methode. 

In [248]:
df  # load Dataframe for review

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [249]:
df.describe() # I describe the dataframe to find some information about Dataframe

,Postal Code,Borough,Neighbourhood
count,180,180,180
unique,180,12,100
top,M4V,Not assigned,Not assigned
freq,1,77,77


In [250]:
df.info() # also .info would be helpful to review the null values, datatype, ...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
Postal Code      180 non-null object
Borough          180 non-null object
Neighbourhood    180 non-null object
dtypes: object(3)
memory usage: 4.3+ KB


<h3>start cleansing the data</h3>
<hr />
<span style='color:skyblue'>as requested by assumpsion we should remove the 'Not assigned' Borough from our records,</span>

In [251]:

Clean_df =df[df['Borough']!='Not assigned']
Clean_df

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


<div style='color:skyblue'> To concatinate Neighbourhood values on same Borough values First i group by the dataframe by Borough then i used transform methode and lambda function on Neighbourhood values.</div>
<hr >


In [252]:
Clean_df['Neighbourhood']= Clean_df.groupby(['Postal Code','Borough'])['Neighbourhood'].transform(lambda x: ','.join(x))
Clean_df =Clean_df.drop_duplicates() # remove duplicate if exist
Clean_df.head()

ipykernel_launcher:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<div style='color:skyblue'>As request, we should replace the 'Not assigned' values on Neighbourhood columns with Borough value </div>

In [253]:
Clean_df['Neighbourhood'].replace('Not assigned',Clean_df['Borough'],inplace=True)
Clean_df.rename(columns={'Postal Code': 'PCode'},inplace=True)
Clean_df

,PCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


<div style='color:green'><h4> The result of cleansing </h4></div>

In [254]:
Clean_df.shape

(103, 3)

In [255]:
geodf = pd.read_csv('Geospatial_Coordinates.csv')

In [256]:
geodf

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [257]:
Clean_df= Clean_df.set_index('PCode').join(geodf.set_index('Postal Code'))
Toronto_df = Clean_df[Clean_df['Borough'].str.contains('Toronto')].reset_index(drop=True)

In [258]:
Toronto_df

,Borough,Neighbourhood,Latitude,Longitude
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,Downtown Toronto,St. James Town,43.651494,-79.375418
4,East Toronto,The Beaches,43.676357,-79.293031
5,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,Downtown Toronto,Christie,43.669542,-79.422564
8,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


In [259]:

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="Tr_explorer")
location = geolocator.geocode(address)
latt = location.latitude
long = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latt, long))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [260]:
city_map = fl.Map(location=[latt, long], zoom_start=10)

In [261]:
for lat, lng, borough, neighborhood in zip(Toronto['Latitude'], Toronto['Longitude'], Toronto['Borough'], Toronto['Neighbourhood']):
    label = '{}, {}'.format(Toronto, borough)
    label = fl.Popup(label, parse_html=True)
    fl.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(city_map)  
city_map

In [262]:
CLIENT_ID = 'SSNWMNIKVK3MUF3S5TYVPHMAPVJNWQV5OKUMCMXKIXY1CJKC' 
CLIENT_SECRET = '4S4OOXG1NGKOTSONE0RTRGGAG05YE5VOIXOTGC4QO0JBTBQQ'
VERSION = '20180605' 
LIMIT = 100 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SSNWMNIKVK3MUF3S5TYVPHMAPVJNWQV5OKUMCMXKIXY1CJKC
CLIENT_SECRET:4S4OOXG1NGKOTSONE0RTRGGAG05YE5VOIXOTGC4QO0JBTBQQ


In [263]:

def getNearbyVenues(names, latitudes, longitudes, radius=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [264]:

Toronto_venues = getNearbyVenues(names=Toronto_df['Neighbourhood'],
                                   latitudes=Toronto_df['Latitude'],
                                   longitudes=Toronto_df['Longitude']
                                  )
Toronto_venues

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Runnymede, The Junction, Weston-Pellam Park, Carlton Village
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harb

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
1,"Regent Park, Harbourfront",43.654260,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
2,"Garden District, Ryerson",43.657162,-79.378937,Ryerson Image Centre,43.657523,-79.379460,Art Gallery
3,"Garden District, Ryerson",43.657162,-79.378937,Balzac's Coffee,43.657854,-79.379200,Coffee Shop
4,St. James Town,43.651494,-79.375418,Gyu-Kaku Japanese BBQ,43.651422,-79.375047,Japanese Restaurant
...,...,...,...,...,...,...,...
98,"First Canadian Place, Underground city",43.648429,-79.382280,Papyrus,43.648195,-79.381392,Gift Shop
99,"First Canadian Place, Underground city",43.648429,-79.382280,Prairie Girl Bakery,43.648332,-79.382305,Cupcake Shop
100,"First Canadian Place, Underground city",43.648429,-79.382280,First Canadian Place,43.648482,-79.382443,Building
101,"First Canadian Place, Underground city",43.648429,-79.382280,Starbucks,43.649028,-79.381593,Coffee Shop


In [265]:
Toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",1,1,1,1,1,1
Central Bay Street,3,3,3,3,3,3
Christie,1,1,1,1,1,1
"Commerce Court, Victoria Hotel",15,15,15,15,15,15
Davisville,9,9,9,9,9,9
"First Canadian Place, Underground city",13,13,13,13,13,13
"Garden District, Ryerson",2,2,2,2,2,2
"Harbourfront East, Union Station, Toronto Islands",3,3,3,3,3,3
"Kensington Market, Chinatown, Grange Park",4,4,4,4,4,4


In [266]:
Torrento_venues.head(10)

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront,Queen's Park, Ontari...",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront,Queen's Park, Ontari...",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront,Queen's Park, Ontari...",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront,Queen's Park, Ontari...",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Regent Park, Harbourfront,Queen's Park, Ontari...",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
5,"Regent Park, Harbourfront,Queen's Park, Ontari...",43.65426,-79.360636,Corktown Common,43.655618,-79.356211,Park
6,"Regent Park, Harbourfront,Queen's Park, Ontari...",43.65426,-79.360636,Dominion Pub and Kitchen,43.656919,-79.358967,Pub
7,"Regent Park, Harbourfront,Queen's Park, Ontari...",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
8,"Regent Park, Harbourfront,Queen's Park, Ontari...",43.65426,-79.360636,The Extension Room,43.653313,-79.359725,Gym / Fitness Center
9,"Regent Park, Harbourfront,Queen's Park, Ontari...",43.65426,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site


<h3>Analyze the data </h3>
<hr />
At first encode the Neighbourhood by category


In [267]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Torrento_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighbourhood'] = Torrento_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Neighbourhood,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,"Regent Park, Harbourfront,Queen's Park, Ontari...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront,Queen's Park, Ontari...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront,Queen's Park, Ontari...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront,Queen's Park, Ontari...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront,Queen's Park, Ontari...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [268]:
toronto_grouped = Toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,"Dufferin, Dovercourt Village,Little Portugal, ...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006410,0.000000,0.006410,...,0.000000,0.000000,0.000000,0.000000,0.012821,0.000000,0.012821,0.006410,0.000000,0.012821
1,"Lawrence Park,Roselawn,Davisville North,Forest...",0.000000,0.000000,0.000000,0.000000,0.000000,0.008772,0.000000,0.000000,0.000000,...,0.000000,0.008772,0.008772,0.000000,0.000000,0.000000,0.008772,0.000000,0.000000,0.008772
2,"Regent Park, Harbourfront,Queen's Park, Ontari...",0.000822,0.000822,0.001645,0.002467,0.001645,0.009868,0.001645,0.004112,0.009046,...,0.000000,0.000000,0.000822,0.002467,0.010691,0.000822,0.003289,0.006579,0.000822,0.005757
3,"Runnymede, The Junction, Weston-Pellam Park, C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"The Beaches,The Danforth West, Riverdale,India...",0.000000,0.000000,0.000000,0.000000,0.000000,0.025424,0.000000,0.000000,0.000000,...,0.008475,0.000000,0.016949,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025424


<h4> Next, calculate the frequency of each venue for each neighbourhood </h4>
<hr />

In [269]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Dufferin, Dovercourt Village,Little Portugal, Trinity,Brockton, Parkdale Village, Exhibition Place,High Park, The Junction South,Parkdale, Roncesvalles,Runnymede, Swansea----
                venue  freq
0                Café  0.07
1                 Bar  0.07
2         Coffee Shop  0.06
3  Italian Restaurant  0.04
4              Bakery  0.04


----Lawrence Park,Roselawn,Davisville North,Forest Hill North & West, Forest Hill Road Park,North Toronto West, Lawrence Park,The Annex, North Midtown, Yorkville,Davisville,Moore Park, Summerhill East,Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park----
            venue  freq
0     Coffee Shop  0.07
1  Sandwich Place  0.07
2            Café  0.06
3            Park  0.04
4     Pizza Place  0.04


----Regent Park, Harbourfront,Queen's Park, Ontario Provincial Government,Garden District, Ryerson,St. James Town,Berczy Park,Central Bay Street,Christie,Richmond, Adelaide, King,Harbourfront East, Union Station, Toronto Islands,Toron

<h4> Create a function to retrive the most common venues for each neighbourhood </h4>
<hr />

In [270]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [271]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Dufferin, Dovercourt Village,Little Portugal, ...",Bar,Café,Coffee Shop,Italian Restaurant,Bakery,Breakfast Spot,Restaurant,Park,Bank,Bookstore
1,"Lawrence Park,Roselawn,Davisville North,Forest...",Sandwich Place,Coffee Shop,Café,Pizza Place,Park,Restaurant,Sushi Restaurant,Pub,Gym,Dessert Shop
2,"Regent Park, Harbourfront,Queen's Park, Ontari...",Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Seafood Restaurant,Sushi Restaurant
3,"Runnymede, The Junction, Weston-Pellam Park, C...",Pizza Place,Grocery Store,Convenience Store,Brewery,Bus Line,Distribution Center,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room
4,"The Beaches,The Danforth West, Riverdale,India...",Greek Restaurant,Coffee Shop,Brewery,Italian Restaurant,Restaurant,Ice Cream Shop,Park,Bookstore,Pub,Café


<h3> Clustering </h3>
<hr />
<h4> Apply the model and visualization </h4>

In [272]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 4, 2, 1, 0])

In [273]:
neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Dufferin, Dovercourt Village,Little Portugal, ...",Bar,Café,Coffee Shop,Italian Restaurant,Bakery,Breakfast Spot,Restaurant,Park,Bank,Bookstore
1,"Lawrence Park,Roselawn,Davisville North,Forest...",Sandwich Place,Coffee Shop,Café,Pizza Place,Park,Restaurant,Sushi Restaurant,Pub,Gym,Dessert Shop
2,"Regent Park, Harbourfront,Queen's Park, Ontari...",Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Seafood Restaurant,Sushi Restaurant
3,"Runnymede, The Junction, Weston-Pellam Park, C...",Pizza Place,Grocery Store,Convenience Store,Brewery,Bus Line,Distribution Center,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room
4,"The Beaches,The Danforth West, Riverdale,India...",Greek Restaurant,Coffee Shop,Brewery,Italian Restaurant,Restaurant,Ice Cream Shop,Park,Bookstore,Pub,Café


In [274]:
if ('Cluster Labels' not in  neighborhoods_venues_sorted.columns):
    neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_) 
Torento_merged = Torento_df

# merge Torento_merged with Torento_df to add latitude/longitude for each neighbourhood
Torento_merged = Torento_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

Torento_merged.head() 

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront,Queen's Park, Ontari...",43.654260,-79.360636,2,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Seafood Restaurant,Sushi Restaurant
1,Downtown Toronto,"Regent Park, Harbourfront,Queen's Park, Ontari...",43.662301,-79.389494,2,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Seafood Restaurant,Sushi Restaurant
2,Downtown Toronto,"Regent Park, Harbourfront,Queen's Park, Ontari...",43.657162,-79.378937,2,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Seafood Restaurant,Sushi Restaurant
3,Downtown Toronto,"Regent Park, Harbourfront,Queen's Park, Ontari...",43.651494,-79.375418,2,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Seafood Restaurant,Sushi Restaurant
4,East Toronto,"The Beaches,The Danforth West, Riverdale,India...",43.676357,-79.293031,0,Greek Restaurant,Coffee Shop,Brewery,Italian Restaurant,Restaurant,Ice Cream Shop,Park,Bookstore,Pub,Café


In [275]:
# create map
map_clusters = fl.Map(location=[latt, long], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Torento_merged['Latitude'], Torento_merged['Longitude'], Torento_merged['Neighbourhood'], Torento_merged['Cluster Labels']):
    label = fl.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    fl.CircleMarker(
        [lat, lon],
        radius=3,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters